In [1]:
%load_ext bigdata
%hive_start
%timeout 300

In [2]:
!hdfs dfs -ls /tmp
!hdfs dfs -mkdir /tmp/wordcount
!hdfs dfs -ls /tmp/

Found 4 items
drwxrwx---   - root supergroup          0 2020-02-18 19:12 /tmp/hadoop-yarn
drwxrwxrwx   - root supergroup          0 2020-02-18 19:19 /tmp/hive
drwxrwxrwx   - root supergroup          0 2020-02-18 20:58 /tmp/output
drwxr-xr-x   - root supergroup          0 2020-02-18 19:38 /tmp/wordcount
mkdir: `/tmp/wordcount': File exists
Found 4 items
drwxrwx---   - root supergroup          0 2020-02-18 19:12 /tmp/hadoop-yarn
drwxrwxrwx   - root supergroup          0 2020-02-18 19:19 /tmp/hive
drwxrwxrwx   - root supergroup          0 2020-02-18 20:58 /tmp/output
drwxr-xr-x   - root supergroup          0 2020-02-18 19:38 /tmp/wordcount


In [3]:
!hdfs dfs -copyFromLocal *.csv  /tmp/wordcount/

copyFromLocal: `/tmp/wordcount/tbl0.csv': File exists
copyFromLocal: `/tmp/wordcount/tbl1.csv': File exists


In [4]:
%%hive

DROP TABLE IF EXISTS tbl0;
DROP TABLE IF EXISTS tbl1;

DROP TABLE IF EXISTS tbl0;
OK
Time taken: 28.573 seconds
DROP TABLE IF EXISTS tbl1;
OK
Time taken: 0.388 seconds


In [5]:
%%hive

CREATE TABLE tbl0 (
    c1 INT,
    c2 STRING,
    c3 INT,
    c4 DATE,
    c5 ARRAY<CHAR(1)>, 
    c6 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
LOAD DATA LOCAL INPATH 'tbl0.csv' INTO TABLE tbl0;

CREATE TABLE tbl0 (
    c1 INT,
    c2 STRING,
    c3 INT,
    c4 DATE,
    c5 ARRAY<CHAR(1)>, 
    c6 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
OK
Time taken: 1.842 seconds
LOAD DATA LOCAL INPATH 'tbl0.csv' INTO TABLE tbl0;
Loading data to table default.tbl0
OK
Time taken: 3.472 seconds


In [6]:
%%hive

CREATE TABLE tbl1 (
    c1 INT,
    c2 INT,
    c3 STRING,
    c4 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
LOAD DATA LOCAL INPATH 'tbl1.csv' INTO TABLE tbl1;

CREATE TABLE tbl1 (
    c1 INT,
    c2 INT,
    c3 STRING,
    c4 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
OK
Time taken: 0.196 seconds
LOAD DATA LOCAL INPATH 'tbl1.csv' INTO TABLE tbl1;
Loading data to table default.tbl1
OK
Time taken: 1.961 seconds


In [13]:
%%hive
DROP TABLE IF EXISTS lets_keys;
CREATE TABLE lets_keys AS
SELECT a.c2, concat_ws(':', collect_set(CAST(a.c1 AS VARCHAR(10))))
FROM (SELECT c2, c1
FROM tbl0) a
GROUP BY a.c2;

DROP TABLE IF EXISTS lets_keys;
OK
Time taken: 0.034 seconds
CREATE TABLE lets_keys AS
SELECT a.c2, concat_ws(':', collect_set(CAST(a.c1 AS VARCHAR(10))))
FROM (SELECT c2, c1
FROM tbl0) a
GROUP BY a.c2;
Query ID = root_20200218211502_099c0b93-239f-4976-9256-36fdc5d60ab2
Total jobs = 1
Launching Job 1 out of 1
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Starting Job = job_1582053079980_0022, Tracking URL = http://fbebe19f3edd:8088/proxy/application_1582053079980_0022/
Kill Command = /usr/local/hadoop/bin/hadoop job  -kill job_1582053079980_0022
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 1
2020-02-18 21:15:17,734 Stage-1 map = 0%,  reduce = 0%
2

In [14]:
%%hive
INSERT OVERWRITE DIRECTORY '/tmp/output'
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
SELECT * FROM lets_keys;

INSERT OVERWRITE DIRECTORY '/tmp/output'
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
SELECT * FROM lets_keys;
Query ID = root_20200218211541_fc432407-1f6f-4047-9479-7e557a594a85
Total jobs = 3
Launching Job 1 out of 3
Number of reduce tasks is set to 0 since there's no reduce operator
Starting Job = job_1582053079980_0023, Tracking URL = http://fbebe19f3edd:8088/proxy/application_1582053079980_0023/
Kill Command = /usr/local/hadoop/bin/hadoop job  -kill job_1582053079980_0023
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 0
2020-02-18 21:15:58,725 Stage-1 map = 0%,  reduce = 0%
2020-02-18 21:16:08,465 Stage-1 map = 100%,  reduce = 0%, Cumulative CPU 3.32 sec
MapReduce Total cumulative CPU time: 3 seconds 320 msec
Ended Job = job_1582053079980_0023
Stage-3 is selected by condition resolver.
Stage-2 is filtered out by condition resolver.
Stage-4 is filtered out by condition resolver.
Moving data to directory hdfs://0.0.0.0:9000/tmp/output/.hive-staging_hi

In [15]:
!hive -S -e 'source output.hql'

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/local/hive/lib/log4j-slf4j-impl-2.6.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/local/hadoop/share/hadoop/common/lib/slf4j-log4j12-1.7.10.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
File: output.hql is not a file.


In [16]:
!hadoop fs -copyToLocal /tmp/output output
!ls output/*

output/000000_0


In [17]:
!cat output/000000_0

A,6
B,7:9:10
C,2:5:8
D,1:3:4
